# COMP90051 Workshop 11b
## Nuclear Power Plant in Stan
***

In this worksheet, we'll revisit the nuclear power plant model from lectures. 
Recall that the model describes the probability of an alarm sounding conditional on the following variables:

- `FA`: whether the alarm is faulty
- `HT`: whether the temperature is high (i.e. reactor is in meltdown)
- `FG`: whether the temperature gauge is faulty
- `HG`: whether the temperature gauge reads high

A plate diagram for the model is given below. 
Note that we've explicitly included the probability tables as parameters in the model. 
We're also taking a more Bayesian approach by putting a prior over the entries in the probability tables.



Our goal is to infer the probability tables based on observations (assuming all variables are observed).
We'll use Stan to do the inference.
Let's begin by importing the packages we need.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  # pretty plotting style
import pystan          # Python interface for STAN
import pandas as pd    # reading CSV
import arviz as az     # plotting functionality for Bayesian inference

## 1. Loading observations
Let's begin by loading the observations.
The `nuclear.csv` file contains 10000 observations for all variables (HT, FA, FG, HG, AS) as rows.
We load the file into a dataframe, and cast the binary observations to 0/1 integers.

In [ ]:
df = pd.read_csv("nuclear.csv")
df = df.astype(int) # cast from bool to int
df.head()

## 2. Model definition
Stan models are specified using the Stan Modelling Language: a custom language with a syntax similar to C.
It's recommended to store models in `.stan` text files.
Our nuclear model is specified in `nuclear.stan`, which we've reproduced below.

In [ ]:
nuclear_code = """
data {
  int<lower=1> N; // number of observations
  // for each of these, 0 means false, 1 means true
  int<lower=0, upper=1> HT[N];  // whether core temperature high
  int<lower=0, upper=1> FG[N];  // whether gauge is faulty
  int<lower=0, upper=1> FA[N];  // whether alarm is faulty
  int<lower=0, upper=1> HG[N];  // whether gauge reads high
  int<lower=0, upper=1> AS[N];  // whether alarm sounds

  real<lower=0> ALPHA; // alpha to use for beta distributions
  real<lower=0> BETA; // beta to use for beta distributions
}

parameters {
  //probability table for each variable
  real<lower=0, upper=1> p_ht;
  real<lower=0, upper=1> p_fg;  
  real<lower=0, upper=1> p_fa;
  real<lower=0, upper=1> p_hg[2,2];
  real<lower=0, upper=1> p_as[2,2];
}

model {
  // priors for probability table parameters
  p_ht ~ beta(ALPHA, BETA);
  p_fg ~ beta(ALPHA, BETA);
  p_fa ~ beta(ALPHA, BETA);
  for (i in 1:2) {
    for (j in 1:2) {
      p_hg[i, j] ~ beta(ALPHA, BETA); 
      p_as[i, j] ~ beta(ALPHA, BETA);
    }
  }
  
  // likelihood of observing data
  for (n in 1:N) {
    HT[n] ~ bernoulli(p_ht);
    FG[n] ~ bernoulli(p_fg);
    FA[n] ~ bernoulli(p_fa);
    // +1 in indexing below because Stan arrays count from 1
    HG[n] ~ bernoulli(p_hg[HT[n]+1, FG[n]+1]);
    AS[n] ~ bernoulli(p_as[FA[n]+1, HG[n]+1]);
  }
}
"""

There are three blocks in this file:

- the *data* block allows us to pass external variables (from R) into Stan. Note that variables are statically typed.
- the *parameters* block defines the sampling space: i.e. the parameters we'd like to infer.
- the *model* block is where we define the posterior distribution.


## 3. Preparing data
When using the R interface for Stan, we must pass the variables in the *data* block as a named list.
The names should match those used in the `.stan` file.
This is easy to do: we simply convert the dataframe to a list, then concatenate it with the `ALPHA`, `BETA` and `N` parameters.

In [ ]:
nuclear_data = df.to_dict(orient='list')
nuclear_data["ALPHA"] = 1
nuclear_data["BETA"] = 1
nuclear_data["N"] = df.shape[0]

## 4. Running Stan
To run inference, we call Stan, passing a reference to our model specification file, in addition to the data from Python.
We also specify the number of iterations to run, which is related to the number of posterior samples we'd like.
Note that not all of the iterations yield usable samples: some of them are discarded during a warmup period (before the Markov chain has converged to the posterior).

In [ ]:
sm = pystan.StanModel(model_code=nuclear_code)
fit = sm.sampling(data=nuclear_data, iter=1000, chains=2)

Calling the `print` function on the output gives us a summary of the posterior distribution for each parameter.

In [ ]:
print(fit)

We can also plot the posterior with Bayesian credible intervals.

In [ ]:
az.plot_forest(fit, kind="forestplot", combined=True, 
               ridgeplot_overlap=1.5, figsize=(7, 6))
plt.show()

In practice, we should inspect trace plots to ensure the Markov chains are mixing well.
However this is beyond the scope of COMP90051.

In [ ]:
az.plot_trace(fit)
plt.show()